In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from pymilvus import connections, CollectionSchema, FieldSchema, DataType, Collection
from umap import UMAP
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import davies_bouldin_score

/Users/genitaabazi/Desktop/Master Thesis Project/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Connect to Attu, Milvus
connections.connect(
    alias="default",
    user="username",
    password="password",
    host="localhost",
    port="19530"
)

In [3]:
# Create fields for Collection Schema
id = FieldSchema(
    name="id",
    dtype=DataType.VARCHAR,
    is_primary=True,
    max_length=200
)

authors = FieldSchema(
    name="authors",
    dtype=DataType.VARCHAR,
    max_length=10000,
    default_value="Unknown"
)

title = FieldSchema(
    name="title",
    dtype=DataType.VARCHAR,
    max_length=10000,
    default_value="Unknown"
)

paper_categories = FieldSchema(
    name="categories",
    dtype=DataType.VARCHAR,
    max_length=10000,
    default_value="Unknown"
)

paper_license = FieldSchema(
    name="license",
    dtype=DataType.VARCHAR,
    max_length=10000,
    default_value="Unknown"
)

abstract = FieldSchema(
    name="abstract",
    dtype=DataType.VARCHAR,
    max_length=10000,
    default_value="Unknown"
)

update_date = FieldSchema(
    name="update_date",
    dtype=DataType.VARCHAR,
    max_length=256,
    default_value="Unknown"
)

merged = FieldSchema(
    name="merged",
    dtype=DataType.VARCHAR,
    max_length=10000,
    default_value="Unknown"
)

merged_embeddings = FieldSchema(
    name="merged_embeddings",
    dtype=DataType.FLOAT_VECTOR,
    dim=768
)

dimension_X = FieldSchema(
    name="dimension_X",
    dtype=DataType.FLOAT,
)

dimension_Y = FieldSchema(
    name="dimension_Y",
    dtype=DataType.FLOAT
)

cluster = FieldSchema(
    name="cluster",
    dtype=DataType.INT64,
)

distance_to_centroid = FieldSchema(
    name="distance_to_centroid",
    dtype=DataType.FLOAT
)

normalized_distance = FieldSchema(
    name="normalized_distance",
    dtype=DataType.FLOAT
)

schema = CollectionSchema(
    fields=[id, title, authors, paper_categories, paper_license, abstract, update_date, merged, merged_embeddings, dimension_X, dimension_Y ,cluster, distance_to_centroid, normalized_distance],
    description="ArXiv Research Papers",
    enable_dynamic_field=True
)

collection_name = "papers"

In [4]:
# Create collection "arXiv"
collection = Collection(
    name=collection_name,
    schema=schema,
    using="default"
)

In [5]:
# Import arXiv data
data = pd.read_csv("./arxiv_research_papers.csv")

In [6]:
# Convert id & license to String
data["id"] = data["id"].astype(str)
data["license"] = data["license"].astype(str)

# Define a function to process the string within the column "merged_embeddings"'s series values
def process_string(s):
    inner_str = s.strip("[]")
    float_list = [float(value) for value in inner_str.split()]
    return float_list

# Apply the function to the series
data["merged_embeddings"] = data["merged_embeddings"].apply(process_string)

# Convert the embeddings into a 2D NumPy array
embeddings = np.array(data["merged_embeddings"].tolist())

In [7]:
# Perform k-means clustering
num_clusters = 6  # Based on the Elbow Method
kmeans = KMeans(n_clusters=num_clusters, random_state=0, n_init="auto")
cluster_labels = kmeans.fit_predict(embeddings)

# Save cluster_labels as column "cluster"
data["cluster"] = cluster_labels

In [8]:
# Calculate the distance of each point to its cluster centroid
centroids = kmeans.cluster_centers_
distances = euclidean_distances(embeddings, centroids)
closest_centroid_distances = distances[np.arange(len(distances)), kmeans.labels_]

# Normalize these distances within each cluster
data['distance_to_centroid'] = closest_centroid_distances

In [9]:
data['normalized_distance'] = data.groupby('cluster')['distance_to_centroid'].transform(lambda x: (x - x.min()) / (x.max() - x.min()))

In [10]:
# Create a UMAP model
umap_model = UMAP(n_neighbors=15, min_dist=0.5, n_components=2, random_state=42)

# Fit and transform the embeddings
umap_result = umap_model.fit_transform(embeddings)

# Save the UMAP dimensions as columns X and Y, respectively
data["dimension_X"] = umap_result[:, 0]
data["dimension_Y"] = umap_result[:, 1]

/Users/genitaabazi/Desktop/Master Thesis Project/venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [11]:
# Insert the data to the collection in Milvus
for index, row in data.iterrows():
    data_dict = {
        'id': row["id"],
        'authors': row["authors"],
        'title': row["title"],
        'categories': row["categories"],
        'license': row["license"],
        'abstract': row["abstract"],
        'update_date': row["update_date"],
        'merged': row["merged"],
        'merged_embeddings': row["merged_embeddings"],
        'dimension_X': row["dimension_X"],
        'dimension_Y': row["dimension_Y"],
        'cluster': row["cluster"],
        'distance_to_centroid': row["distance_to_centroid"],
        'normalized_distance': row["normalized_distance"]
    }

    collection.insert(data_dict)

collection.flush()

In [12]:
# Create an index for each embedding in the collection
index = {
    'index_type': "IVF_FLAT",
    'metric_type': "L2",
    'params': {"nlist": 1024},
}

collection.create_index("merged_embeddings", index)

# Load the collection
collection.load()

In [13]:
# Define search parameters
search_params = {
    'metric_type': "L2",
    'ignore_growing': False, 
    'params': {'nprobe': 10}
}

In [ ]:
results = collection.search(
    data=[[-0.26960688829421997, 0.7992581129074097, -0.1395236849784851, -0.9237047433853149, 0.6558606624603271, 0.5106121301651001, 1.530570387840271, -0.160568967461586, -1.0542361736297607, 0.12624619901180267, 0.7548688650131226, 0.055621881037950516, -0.578356146812439, 0.5623791217803955, -0.3651939034461975, 1.1950969696044922, -0.8690117597579956, 0.6163325905799866, 1.7254664897918701, -0.08778470754623413, -0.104036346077919, 0.5846306085586548, -0.7327515482902527, -0.3169822692871094, 0.8970857858657837, 0.4825419783592224, -0.30830085277557373, 0.5067426562309265, -0.24710853397846222, 0.8863217234611511, -0.3123522400856018, -0.16555550694465637, -0.8797181248664856, 0.011404309421777725, 0.707604169845581, -0.32548534870147705, 0.117375448346138, 0.10684974491596222, -0.48379015922546387, -0.0962933599948883, -0.5993056893348694, 0.26972389221191406, 1.0549250841140747, -1.265942096710205, 0.45752742886543274, 0.37503883242607117, 0.38919907808303833, 0.5076065063476562, -0.4613163471221924, -0.6175664663314819, -0.5574555397033691, 0.174716055393219, 0.2469581514596939, 2.0996408462524414, 0.8961137533187866, 0.455360472202301, -0.20925462245941162, 0.5270304083824158, -0.09938353300094604, -0.6292788982391357, -0.4377608895301819, -0.20316535234451294, -0.3736587166786194, 0.45263391733169556, 0.5346543788909912, 0.05148102343082428, -0.1636987030506134, 0.5000251531600952, 0.09185481816530228, 1.0475554466247559, 1.0049030780792236, -1.0930002927780151, -0.4215353727340698, 0.7862288951873779, 0.5497628450393677, -0.08470059931278229, 0.001224420964717865, 0.30793026089668274, -0.5376694202423096, -0.15938404202461243, 0.022351764142513275, -0.5063823461532593, -0.3398674428462982, -0.5238789319992065, -1.0834052562713623, 0.6469853520393372, -0.6082175374031067, 0.49501264095306396, 0.19141846895217896, 0.004212282598018646, 1.0996205806732178, 0.020473415032029152, -0.3066827356815338, 0.002201855182647705, -0.4700908064842224, 0.23535306751728058, -0.40855327248573303, 0.05231928080320358, 0.31829434633255005, 1.1849377155303955, -0.977220356464386, 0.8365742564201355, -0.04970315843820572, 0.4888914227485657, 1.2281370162963867, 0.8108776211738586, 0.5248963236808777, -0.7424434423446655, -0.41253483295440674, -0.32940393686294556, -0.5154240727424622, -0.9631332755088806, -0.2908635139465332, -0.8301347494125366, 0.27894341945648193, -0.8821464776992798, -0.5599988102912903, -0.254145085811615, -1.504381775856018, 1.195953607559204, -0.13356593251228333, -0.6609955430030823, 0.10833315551280975, -1.1868340969085693, 0.6303449273109436, 1.0446689128875732, 0.6093005537986755, -0.8669549822807312, 0.585567831993103, -0.022410836070775986, 0.39281052350997925, -0.4726395010948181, 0.47846585512161255, -0.10669787228107452, 0.40215474367141724, -0.3031757175922394, -1.3910200595855713, -1.170224905014038, -0.3285605013370514, -0.9682283401489258, -0.6360872983932495, 0.4481548070907593, -0.3681522309780121, 0.8854456543922424, -0.8480197787284851, -0.052017275243997574, -0.36896055936813354, 0.05020901933312416, -0.563377857208252, -0.6836791038513184, 0.1152750700712204, -0.4602617621421814, -0.22385385632514954, 0.5148532390594482, -0.07140305638313293, 0.2462742030620575, -0.7565501928329468, -0.6129100322723389, -0.7428773045539856, 0.12974122166633606, -0.1660434752702713, -0.4279440641403198, 0.8423407077789307, 0.06755557656288147, -0.41519880294799805, 0.2476981282234192, -0.5935972929000854, -0.0748443454504013, -0.25445082783699036, 0.15212880074977875, -0.7439366579055786, -1.788800835609436, -0.3331031799316406, 0.7158840894699097, 0.8195991516113281, -1.2205063104629517, -0.5179979205131531, 0.21426743268966675, 0.3750000596046448, 0.057910241186618805, -0.9558185338973999, 1.2009990215301514, -0.21514630317687988, -0.2765311896800995, -0.3882632553577423, -0.4538258910179138, 0.20596981048583984, -0.3508870005607605, -1.1136939525604248, -0.15239378809928894, 0.8312886953353882, -0.712746798992157, 0.8516354560852051, -0.6841341853141785, 0.16944725811481476, -0.42818963527679443, -1.043227195739746, -0.728821337223053, -0.2932349443435669, -0.5518088340759277, -0.7749924063682556, 0.50278240442276, 0.08699008822441101, -1.4183313846588135, 0.1610051691532135, -0.4437538683414459, -0.6179701089859009, -1.4230716228485107, -0.03477601706981659, 0.2290082722902298, 0.28013062477111816, -1.0595182180404663, -0.5596044063568115, -0.12892967462539673, -0.17636063694953918, 0.6123772859573364, -0.46521061658859253, 0.28808459639549255, -0.4629572629928589, -0.17159196734428406, -0.42691802978515625, 0.7468023300170898, 0.30184298753738403, 0.27307045459747314, -0.4894227981567383, 1.260635495185852, -0.49196383357048035, 0.04434390366077423, -0.7866430282592773, -0.7658247947692871, 0.2810623347759247, 0.833614706993103, -0.2820669412612915, -0.11550286412239075, -0.08245903253555298, -0.10383456945419312, -0.5964155197143555, 0.872496485710144, 0.016693206503987312, 0.26024922728538513, 0.6437780261039734, 0.6959953308105469, -0.9388717412948608, 0.17231181263923645, -0.15270599722862244, -1.0028660297393799, 0.00587473064661026, 0.22114136815071106, 0.5880777835845947, 0.6266211271286011, -0.8848998546600342, 0.3221583962440491, 0.030784880742430687, -0.3890514075756073, -0.031836770474910736, 0.0783691480755806, 0.4879382252693176, 0.8754499554634094, -0.4715368151664734, -0.6004032492637634, 0.039733484387397766, -0.4462771713733673, 1.4569107294082642, 0.6300350427627563, -0.1690080761909485, -0.08324103057384491, -0.0032660961151123047, -0.22126030921936035, -0.4464573264122009, 0.6697723269462585, -0.05533583462238312, 0.23185676336288452, -0.7741193175315857, -0.34970319271087646, 0.870497465133667, 0.5869742631912231, 0.037111639976501465, -0.8563776016235352, 0.369748055934906, -0.7398361563682556, 0.12054169923067093, -0.7224851846694946, -0.023464731872081757, 0.7121216058731079, -1.5280447006225586, -0.28850769996643066, -0.947085976600647, 0.21579080820083618, -0.8396668434143066, -0.7526298761367798, 2.0647053718566895, 0.14684462547302246, 0.5970437526702881, -0.15904726088047028, -0.17663779854774475, -0.9892604947090149, -1.3799726963043213, 0.4072452485561371, 0.9668287038803101, 0.22993183135986328, -0.3936213552951813, 0.15669217705726624, 0.46570461988449097, -0.5661019086837769, -0.11209393292665482, -1.0434114933013916, 0.24055469036102295, -0.9990670084953308, 0.20619134604930878, -0.29923319816589355, -1.2152934074401855, -1.4119222164154053, 0.21994344890117645, 0.7728981375694275, -0.07569178193807602, 0.5860369205474854, 0.3118942677974701, 0.3642351031303406, 0.6875917315483093, -0.006746433675289154, -0.30458536744117737, -0.8375858068466187, 0.06234082207083702, -1.0924519300460815, 0.2665625810623169, 0.38765522837638855, -0.3034731149673462, 0.6752422451972961, -0.22139982879161835, 1.2437169551849365, -0.43422961235046387, -0.5590720176696777, 0.20372743904590607, -0.20238599181175232, -0.1519303172826767, 0.18260407447814941, 0.040670059621334076, -1.0166195631027222, -0.0018429197371006012, -0.9541943073272705, -0.23941007256507874, 0.9435033798217773, 0.11605186015367508, 0.14376413822174072, 0.08943041414022446, -0.25648248195648193, 0.3170414865016937, 0.13660810887813568, -0.2980550229549408, 0.2961173355579376, -0.3036479949951172, -0.7834305167198181, 0.21089576184749603, -0.2418476790189743, -0.9876176714897156, -0.49589863419532776, 0.4430713653564453, -0.3903863728046417, 1.0281621217727661, 0.36447855830192566, -0.13820043206214905, -0.9601330161094666, -0.12279573082923889, 0.4084599018096924, 1.2327196598052979, -0.0027564596384763718, 1.38566255569458, -0.472445547580719, -0.24592727422714233, 0.14487087726593018, 0.21465864777565002, 0.06050493195652962, -0.8304219841957092, 1.5944089889526367, 0.6251909732818604, -0.8259938955307007, -0.31881001591682434, -0.5288876891136169, -0.398701936006546, 1.1258620023727417, 0.6104284524917603, 0.8682656288146973, -0.8560315370559692, -0.19236958026885986, 0.44930943846702576, -0.2236923724412918, 0.9662549495697021, 0.24777908623218536, 0.6890529990196228, 1.2419743537902832, -0.7737553715705872, -0.8736850023269653, 0.7488776445388794, 1.0344182252883911, 0.02583806961774826, -0.6071993708610535, 0.3481045961380005, 0.366070032119751, 0.5516169667243958, 1.4791523218154907, 0.4507456421852112, -0.633108913898468, -1.8127387762069702, -0.02983558177947998, 0.0021778177469968796, 0.34325283765792847, -0.3692706823348999, 1.948781967163086, 0.7139904499053955, 0.4880225360393524, 0.807321310043335, 0.1959039270877838, -0.5714935660362244, 0.5259267687797546, -0.5739365220069885, -1.0636742115020752, -0.09832055121660233, 0.3109717071056366, 0.046639442443847656, 0.5878056287765503, -0.2867731750011444, 0.43588879704475403, 13.437887191772461, 0.4801102876663208, -0.49242860078811646, -0.2184268683195114, -0.15669533610343933, -0.09818537533283234, -0.39206379652023315, -0.518075704574585, -1.3695497512817383, 0.10858330875635147, 0.8121772408485413, 0.48676660656929016, 0.4353456497192383, 0.13313919305801392, 0.7456783056259155, 0.9628346562385559, -0.6905946731567383, 0.21901041269302368, 0.23802848160266876, 0.385692298412323, 1.1140528917312622, 0.6387341022491455, -1.53145170211792, 1.2931482791900635, 0.2098972499370575, 0.9745583534240723, 0.6033895015716553, -0.7380677461624146, 0.12134850025177002, -0.2041967362165451, 0.7764952182769775, 0.4293833374977112, 0.036620158702135086, 0.4588622450828552, 0.23012098670005798, -0.15749886631965637, 1.302818775177002, 0.2731242775917053, -0.034787509590387344, -0.4860641062259674, 0.012231405824422836, 0.40327346324920654, -0.5959699153900146, -0.0796981230378151, -0.9697025418281555, 1.0148334503173828, 0.44772520661354065, 1.2028217315673828, -0.34261631965637207, -0.6346825361251831, -0.825647234916687, 1.2339786291122437, 0.4335060715675354, 0.08268900960683823, -0.17256572842597961, -0.3875356614589691, 0.9834781289100647, -0.45061004161834717, -0.8175234794616699, 0.8149398565292358, -0.11042900383472443, -0.7634567022323608, 0.45332083106040955, 0.2693086266517639, -0.7363563776016235, 0.4263964593410492, -0.2974924147129059, -0.7329995632171631, 0.018155202269554138, 0.0424145832657814, -0.9549838304519653, -0.873641312122345, -0.32201287150382996, -0.04309045523405075, -0.5410590171813965, -0.35953661799430847, -0.35265302658081055, -0.901114821434021, 0.10426457971334457, -0.9407596588134766, 0.30499714612960815, -0.3224784731864929, 0.10921569913625717, 1.337429404258728, -0.4094785451889038, -0.40626487135887146, -0.02259942889213562, -0.31102654337882996, 0.9272165298461914, 0.5200762152671814, -0.8462401628494263, -1.2341408729553223, 0.6698276400566101, -0.17296484112739563, -0.41390112042427063, -0.04313458502292633, 0.28740546107292175, 0.06485942006111145, 0.32654011249542236, -0.37643131613731384, 0.471300333738327, 0.4009554386138916, 0.04046015068888664, -0.401229590177536, 0.5315763354301453, 0.3399764597415924, 0.7399939298629761, 0.018905997276306152, 0.43624672293663025, 0.22709569334983826, 0.2892633378505707, -0.3578398525714874, 0.433030366897583, -0.411834180355072, -0.43839094042778015, -1.0941624641418457, -0.8147889375686646, 0.1689998209476471, -0.36846429109573364, -0.09277258068323135, 0.2593732178211212, 0.5478106141090393, 0.3846581280231476, 0.05875264108181, 0.7480764389038086, -0.6392328143119812, 0.3169972002506256, -0.21162879467010498, 0.4886513948440552, -0.2965938448905945, -0.03791194409132004, 0.22147685289382935, 0.08455855399370193, 0.2003466933965683, 0.21872982382774353, 0.8325552940368652, -0.0442938357591629, -0.25455227494239807, 0.33616700768470764, -0.07070502638816833, -0.8934202194213867, -0.6780246496200562, -0.7471179962158203, -0.052172087132930756, -1.0101940631866455, -0.16146166622638702, 0.4772294759750366, 0.21513046324253082, -0.055052582174539566, -0.08419989049434662, 0.2903571128845215, 0.12484017014503479, 0.17092418670654297, -0.2898264527320862, 0.45971429347991943, 0.011904612183570862, 0.3790077865123749, -0.6890631318092346, -0.20164038240909576, 0.6911900043487549, 0.8172625303268433, 0.5814129710197449, -0.4230995178222656, 0.0317019522190094, 0.808516263961792, -0.6249566674232483, -0.5864009857177734, -0.31143513321876526, 0.25751206278800964, -0.5895199775695801, 0.1598024219274521, 0.512028157711029, -0.1931169182062149, -0.2075849026441574, -0.20814448595046997, -0.3736828863620758, 0.7600356340408325, 0.2671460509300232, -0.05701947957277298, -0.32728666067123413, -0.49519383907318115, -0.19584500789642334, -0.5455167889595032, 0.9124071598052979, 1.2910854816436768, 0.10886023938655853, -0.3580056428909302, -0.34127140045166016, 0.3031995892524719, 0.07737180590629578, -0.05593321472406387, -0.2042086273431778, -0.8755732774734497, -1.8621501922607422, -0.3689752221107483, -0.08273852616548538, -1.1953341960906982, -0.002994813024997711, -0.04940593242645264, -0.2261572927236557, 0.24520981311798096, 0.07506163418292999, 1.091156005859375, 0.9098528623580933, -0.6337268352508545, 0.4800363779067993, -0.360195130109787, 0.4243694543838501, 0.028251364827156067, 0.10752931237220764, -0.4108375310897827, 0.5384889841079712, -0.9039819240570068, 0.10677768290042877, -0.8969284892082214, 0.5791189074516296, -0.7683695554733276, 0.29439887404441833, -0.07376125454902649, -0.7477649450302124, 0.04596659913659096, -0.5833514332771301, -0.5174842476844788, 0.09892749786376953, 0.8630989789962769, 1.266158938407898, -0.14535358548164368, 0.5190197825431824, -0.09862766414880753, -0.30540743470191956, 0.07580354809761047, -1.307796835899353, -0.7852518558502197, 0.2227010726928711, -0.6169676780700684, 1.2530312538146973, 0.2941529154777527, 0.5428722500801086, -0.983190655708313, 0.2822994291782379, 1.4518382549285889, -0.11165362596511841, 1.0059702396392822, 0.4764987528324127, -0.973197340965271, -1.0008859634399414, -0.3215966522693634, -1.048492193222046, -1.3783442974090576, -0.6770458221435547, 0.6163373589515686, 0.44279196858406067, 0.25272583961486816, -0.9927358627319336, -0.907810628414154, -1.3563390970230103, -0.15907442569732666, -0.895633339881897, 0.6345207095146179, 0.030979320406913757, 0.2782573103904724, 1.462791919708252, -0.3999650478363037, -0.5818812847137451, -0.17994742095470428, 0.1996632218360901, 1.1389909982681274, 0.03474020957946777, 0.23702165484428406, -1.0125136375427246, 0.24377867579460144, 0.9465748071670532, -0.04830733314156532, 0.5046910643577576, 0.46390584111213684, -0.6608723402023315, 0.7847814559936523, -0.5383269786834717, 1.5474961996078491, 0.008471443317830563, -0.7146462798118591, 1.4458506107330322, 0.3489494323730469, -0.783378541469574, -0.6041088104248047, -0.9999672174453735, 0.17296400666236877, 0.3800385892391205, -0.7095357179641724, 0.06287777423858643, 0.9828124046325684, -0.6315118074417114, -0.3046766519546509, 0.5087835788726807, -0.49326932430267334, -0.05724719911813736, -0.24179711937904358, -0.07612036913633347, 0.8114503026008606, 0.5324393510818481, 0.27039867639541626, -0.32001006603240967, -0.019727887585759163, 0.6384180784225464, 0.093159981071949, 0.9840837717056274, 0.9400886297225952, -1.691635251045227, 0.470960408449173, -0.43120935559272766, 0.28423017263412476, 0.11559551954269409, 0.3181120455265045, 1.0407016277313232, -0.05006398260593414, 0.5487536191940308, 0.2661888003349304, 0.6059353351593018, 1.2162315845489502, 0.46793097257614136, 0.7236024141311646, -1.6248879432678223, 0.130024254322052, -0.3484870195388794, 0.4516637325286865, 0.2928507924079895, 0.1135227233171463, -0.6350952982902527, -0.6868001222610474, -0.25594812631607056, 0.4927177131175995, -0.19853997230529785, 0.4163479804992676, 1.4675415754318237, 0.7616264224052429, -0.33875322341918945, 0.35791468620300293, -0.06995626538991928, 0.0052932873368263245, -1.0394576787948608, -0.7205036282539368, 0.02730463817715645, -0.7332592606544495, -0.18608063459396362, 0.2307729423046112, 0.11260438710451126]],
    anns_field="merged_embeddings", 
    param=search_params,
    limit=10,
    expr=None,
    # Set the names of the fields you want to retrieve from the search result
    output_fields=["abstract"],
    consistency_level="Strong"
)

In [ ]:
# Get the abstracts of all returned hits
[r.entity.get("abstract") for r in results[0]]

In [14]:
# Calculate the distance of each point to its cluster centroid
# centroids = kmeans.cluster_centers_
# distances = euclidean_distances(embeddings, centroids)
# closest_centroid_distances = distances[np.arange(len(distances)), kmeans.labels_]
# 
# Normalize these distances within each cluster
# data['distance_to_centroid'] = closest_centroid_distances
# data['normalized_distance'] = data.groupby('cluster')['distance_to_centroid'].transform(lambda x: (x - x.min()) / (x.max() - x.min()))

# Create a DataFrame for plotting
df = pd.DataFrame({
    'X': data["dimension_X"],  # UMAP dimension 1
    'Y': data["dimension_Y"],  # UMAP dimension 2
    'cluster': data["cluster"],
    'normalized_distance': data['normalized_distance'],
    'title': data['title'],
    'category': data['categories']
})

df['marker_size'] = df['normalized_distance'] * 10 + 5  # Ensures that the marker size is never zero

# Create a 2D scatter plot using Plotly Express
fig = px.scatter(
    df,
    x="X",
    y="Y",
    color="cluster",  # Different color for each cluster
    size="marker_size",  # Marker size based on distance to centroid
    color_continuous_scale='Viridis',  # Choose a color scale
    title="UMAP 2D Visualization with Cluster Identity and Hierarchical Coloring",
    hover_data=['title', 'category']
)

# Update layout
fig.update_layout(
    xaxis_title="UMAP Dimension 1",
    yaxis_title="UMAP Dimension 2",
    paper_bgcolor='white',
    plot_bgcolor='white',
    showlegend=True,
    xaxis=dict(showgrid=False, zeroline=False, visible=False),  # Hide X-axis
    yaxis=dict(showgrid=False, zeroline=False, visible=False),  # Hide Y-axis
    margin=dict(l=0, r=0, b=0, t=0),
    title=dict(x=0.5)  # Center the title
)

# Save the plot
fig.write_html("umap_2d_clusters_weighted.html")

In [ ]:
# Evaluate the performance of the clustering algorithm using a Silhouette score
silhouette_score(embeddings, cluster_labels, metric="euclidean")

# 2 0.08840787510295803
# 3 0.08528259860308735
# 4 0.08498083339871042
# 5 0.08930270882457458
# 6 0.08315755278555233
# 7 0.08754382920650787
# 8 0.07860793105947075
# 9 0.07037339998948082

In [ ]:
db_index = davies_bouldin_score(embeddings, cluster_labels)

db_index
# 2 3.2959510866219603
# 3 2.897708431590098
# 4 2.9548232073703318
# 5 2.7487997957737007
# 6 3.0145723072976307
# 7 2.85874385775526
# 8 2.8639277228524183
# 9 2.943365772086929

In [ ]:
# Create a DataFrame from the results
df = pd.DataFrame({
    'X': data["dimension_X"],
    'Y': data["dimension_Y"],
    'cluster': cluster_labels,
    'title': data["title"],
    'category': data["categories"]
})

# Create a 2D scatter plot using Plotly Express
fig = px.scatter(
    df,
    x="X",
    y="Y",
    title="UMAP 2D Visualization with K-Means Clustering",
    hover_data=["title", "category"],
    color=df['cluster'].astype(str)  # This converts the labels to string, making the color scale discrete.
)

fig.update_layout(
    title="UMAP 2D Visualization with K-Means Clustering",
    xaxis_title="UMAP Dimension 1",
    yaxis_title="UMAP Dimension 2",
    paper_bgcolor='white',  # Set the background color of the paper to white
    plot_bgcolor='white',   # Set the background color of the plot area to white
    showlegend=False,       # Remove the legend
    xaxis=dict(showgrid=False, showline=False, zeroline=False),  # Update x-axis properties
    yaxis=dict(showgrid=False, showline=False, zeroline=False),  # Update y-axis properties
    margin=dict(l=0, r=0, b=0, t=0)
)

# Save the plot as an HTML file
fig.write_html("umap_2d_kmeans_clusters.html")